In [1]:
import pandas as pd

In [8]:
df_train = pd.read_csv('data/감성분석/ratings_train.txt', sep='\t')  #encoding='utf-8"
df_train.head()  #5~8:1, 1~4:0

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
df_train.shape

(150000, 3)

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [13]:
df_train.isnull().sum()

id          0
document    5
label       0
dtype: int64

In [15]:
filt = df_train['document'].notnull()
df_train = df_train[filt]
df_train.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [17]:
df_train['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [ ]:
#특수문자 빼고 한글만 남기기 - 정규화
import re
pattern = r'[^가-힣]+'  #한글이 아닌 패턴
df_train['document']=df_train['document'].apply(lambda x: re.sub(pattern, ' ', x))

In [19]:
df_train.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [22]:
df_test = pd.read_csv('data/감성분석/ratings_test.txt', sep='\t')
df_test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [23]:
df_test.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [24]:
filt = df_test['document'].notnull()
df_test = df_test[filt]
df_test.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [25]:
#특수문자 빼고 한글만 남기기 - 정규화
df_test['document']=df_test['document'].apply(lambda x: re.sub(pattern, ' ', x))

In [26]:
df_test.head()

,id,document,label
0,6270596,굳,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


In [29]:
#csv로 저장
df_train.to_csv('data/감성분석/train.csv', index=False)
df_test.to_csv('data/감성분석/test.csv', index=False)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer #문자를 숫자로 벡터작업

In [32]:
text = ['I go to my home my home is very large',
        'I went out my home I go to the market',
        'I bought a yellow lemon I go back to home']

In [46]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 3),
    min_df=2,
    max_df=0.9
)
tfidf.fit(text)
tfidf_text = tfidf.transform(text)
sorted(tfidf.vocabulary_.items())

[('go to', 0), ('my', 1), ('my home', 2)]

In [48]:
#한글 형태소 분리작업
from konlpy.tag import Okt
okt = Okt()

In [50]:
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [52]:
text = '즐거운 추석 명절을 건강히 잘 보내세요!'
okt_tokenizer(text)

['즐거운', '추석', '명절', '을', '건강히', '잘', '보내세요', '!']

In [54]:
tfidf = TfidfVectorizer(
    tokenizer=okt_tokenizer, #한글 형태소 분리
    ngram_range=(1, 2),
    min_df=3,  #3개 미만 제거
    max_df=0.9  #90%이상 반복 단어 제거
)

In [55]:
# df_train=df_train[:1000]
tfidf.fit(df_train['document'])
tfidf_train = tfidf.transform(df_train['document'])

c:\Users\AIPM2\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# sorted(tfidf.vocabulary_.items())

[('가', 0),
 ('가 가고', 1),
 ('가 가끔', 2),
 ('가 가네요', 3),
 ('가 가는', 4),
 ('가 가는데', 5),
 ('가 가득', 6),
 ('가 가득한', 7),
 ('가 가면', 8),
 ('가 가미', 9),
 ('가 가슴', 10),
 ('가 가장', 11),
 ('가 가족', 12),
 ('가 가지', 13),
 ('가 가지는', 14),
 ('가 가진', 15),
 ('가 가질', 16),
 ('가 각본', 17),
 ('가 간다', 18),
 ('가 간닷', 19),
 ('가 갈리는', 20),
 ('가 갈릴', 21),
 ('가 갈릴듯', 22),
 ('가 갈수록', 23),
 ('가 감독', 24),
 ('가 감동', 25),
 ('가 감명', 26),
 ('가 갑', 27),
 ('가 갑자기', 28),
 ('가 강', 29),
 ('가 강한', 30),
 ('가 갖고', 31),
 ('가 같고', 32),
 ('가 같은', 33),
 ('가 같이', 34),
 ('가 개', 35),
 ('가 개념', 36),
 ('가 개봉', 37),
 ('가 개성', 38),
 ('가 개연', 39),
 ('가 개인', 40),
 ('가 개판', 41),
 ('가 걍', 42),
 ('가 거기', 43),
 ('가 거듭', 44),
 ('가 거의', 45),
 ('가 거지', 46),
 ('가 걱정', 47),
 ('가 걸작', 48),
 ('가 겁', 49),
 ('가 게이', 50),
 ('가 게임', 51),
 ('가 겨우', 52),
 ('가 결국', 53),
 ('가 결말', 54),
 ('가 결여', 55),
 ('가 결합', 56),
 ('가 결혼', 57),
 ('가 경', 58),
 ('가 경찰', 59),
 ('가 계속', 60),
 ('가 고갈', 61),
 ('가 고생', 62),
 ('가 고스', 63),
 ('가 고통', 64),
 ('가 곧', 65),
 ('가 공감', 66),
 ('가 공존

In [56]:
tfidf_train.shape

(149995, 113814)

In [58]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=1, max_iter=500, random_state=0)

In [63]:
model.fit(tfidf_train, df_train['label'])

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,0
,solver,'lbfgs'
,max_iter,500
,multi_class,'deprecated'


In [65]:
from sklearn.model_selection import GridSearchCV
params = {
    'C':[0.001, 0.01, 0.1, 1, 10],
    'max_iter':[50, 100, 300, 500],
}

In [66]:
grid = GridSearchCV(model, param_grid=params, scoring='accuracy', cv=3, verbose=1)

In [69]:
import warnings
warnings.filterwarnings('ignore')
grid.fit(tfidf_train, df_train['label'])

Fitting 3 folds for each of 20 candidates, totalling 60 fits


,estimator,LogisticRegre...andom_state=0)
,param_grid,"{'C': [0.001, 0.01, ...], 'max_iter': [50, 100, ...]}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [71]:
grid.best_params_, grid.best_score_

({'C': 10, 'max_iter': 100}, 0.8513750358210492)

In [72]:
best_model = grid.best_estimator_

In [75]:
train_pred = best_model.predict(tfidf_train)
train_pred[:50]

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1], dtype=int64)

In [76]:
df_train['prediction'] = train_pred

In [77]:
df_train.head()

,id,document,label,prediction
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1,1
2,10265843,너무재밓었다그래서보는것을추천한다,0,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,1


In [79]:
filt = df_train['label']==df_train['prediction']
df_equal = df_train[filt]

In [83]:
df_equal.shape[0], df_train.shape[0], df_equal.shape[0]/ df_train.shape[0]

(143221, 149995, 0.9548384946164872)

In [86]:
df_test.head()

,id,document,label
0,6270596,굳,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


In [89]:
tfidf_test = tfidf.transform(df_test['document'])

In [90]:
test_pred = best_model.predict(tfidf_test)

In [91]:
df_test['prediction'] = test_pred

In [92]:
df_test.head()

,id,document,label,prediction
0,6270596,굳,1,1
1,9274899,,0,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,0


In [93]:
filt = df_test['label'] == df_test['prediction']
df_equal = df_test[filt]

In [95]:
len(df_test), len(df_equal), len(df_equal)/len(df_test)

(49997, 42762, 0.8552913174790487)

In [ ]:
text = '영화가 너무 재미있어요.ㅎㅎㅎ^^ 주인공들도 연기를 참 잘해요~~~~~~'
find_text = re.findall(r'[가-힣]+', text)
text_join = [' '.join(find_text)]
tfidf_text = tfidf.transform(text_join)

In [105]:
best_model.predict(tfidf_text)

array([1], dtype=int64)

In [107]:
text = '영화가 너무 구리네요.ㅎㅎㅎ^^ 주인공들도 연기를 참~~~~~~'
find_text = re.findall(r'[가-힣]+', text)
text_join = [' '.join(find_text)]
tfidf_text = tfidf.transform(text_join)
best_model.predict(tfidf_text)

array([0], dtype=int64)

In [117]:
def predict(text):
    find_text = re.findall(r'[가-힣]+', text)
    text_join = [' '.join(find_text)]
    tfidf_text = tfidf.transform(text_join)

    result = best_model.predict(tfidf_text)
    if result[0]==0:
        return "부정"
    
    else:
        return "긍정"

In [118]:
text = '영화가 너무 구려요.ㅎㅎㅎ^^ 주인공들도 연기를 참~~~~'
predict(text)

'부정'

In [138]:
text = '아이와 함께 보기는 어려울거 같아요'
predict(text)

'긍정'

In [152]:
def createModel():
    #학습데이터 생성
    import pandas as pd
    df_train = pd.read_csv('data/감성분석/train.csv')
    df_train.head()
    df_train = df_train[:1000]

    #한글 형태소 분리 함수
    from konlpy.tag import Okt
    okt = Okt()
    def okt_tokenizer(text):
        tokens = okt.morphs(text)
        return tokens

    #document 벡터화
    from sklearn.feature_extraction.text import TfidfVectorizer
    vector = TfidfVectorizer(
        tokenizer=okt_tokenizer, 
        ngram_range=(1, 2), 
        min_df=3, 
        max_df=0.9,
        token_pattern=None
    )
    vector.fit(df_train['document'])
    vector_trans = vector.transform(df_train['document'])

    #모델생성 후 모델학습
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression(C=10, max_iter=100)
    model.fit(vector_trans, df_train['label'])

    return vector, model

In [153]:
vector, model = createModel()

In [ ]:
import re

text='200억대 횡령·배임 혐의를 받는 조현범 한국앤컴퍼니 그룹 회장이 1심에서... 한국타이어앤테크놀로지 자금으로 지급하고, 회사 소유의 가구 2개를 자신의 주거지에서 사용한 혐의(2억6000만원)...'
find_text = re.findall(r'[가-힣]+', text)
text_join = [' '.join(find_text)]

vector_text=vector.transform(text_join)
predict = model.predict(vector_text)
print(predict)

[0]
